# Identificação de Homólogos

In [12]:
import os
from Bio import SeqIO
from Bio.Blast import NCBIWWW, NCBIXML
from scripts_python import BLASTer

a = %pwd
wd = (a.rsplit('/',2))[0]
seq_id = "NM_004335"


In [13]:

"""record = SeqIO.read(os.path.join( wd,"data/homologue", f"{seq_id}_ORF7a.fasta"), "fasta")

result_handle = NCBIWWW.qblast("blastn", "nr", record.format("fasta"), short_query=True, expect=0.10, word_size=7, entrez_query='all[filter] NOT "Severe acute respiratory syndrome coronavirus 2"[Organism]', )

save_file = open(os.path.join( wd,"data/reference", f"{seq_id}_ORF7a_blast.xml", "w"))
save_file.write(result_handle.read())
save_file.close()
result_handle.close()"""

'record = SeqIO.read(os.path.join( wd,"data/homologue", f"{seq_id}_ORF7a.fasta"), "fasta")\n\nresult_handle = NCBIWWW.qblast("blastn", "nr", record.format("fasta"), short_query=True, expect=0.10, word_size=7, entrez_query=\'all[filter] NOT "Severe acute respiratory syndrome coronavirus 2"[Organism]\', )\n\nsave_file = open(os.path.join( wd,"data/reference", f"{seq_id}_ORF7a_blast.xml", "w"))\nsave_file.write(result_handle.read())\nsave_file.close()\nresult_handle.close()'

## Carregar ficheiro XML da pesquisa

In [14]:
"""result_handle= open(os.path.join( wd,"data/homologue/BST2", "0069KN2K01R-Alignment.xml"))
blast_record_n = NCBIXML.read(result_handle)
result_handle.close()

print('Foram carregadas 100 seq:', len(blast_record_n.alignments))"""

result_handle= open(os.path.join( wd,"data/homologue/BST2/00D0DT8J016-Alignment.xml"))
blast_record_p = NCBIXML.read(result_handle)
result_handle.close()

print('Foram carregadas 100 seq:', len(blast_record_p.alignments))

Foram carregadas 100 seq: 100


## Identificar Homologos

In [15]:
from Bio.Seq import Seq
from Bio.SeqRecord import SeqRecord
from Bio import SeqIO

query_len = blast_record_p.query_length

homologue = []

print("Accession", "Identity%")
for aln in blast_record_p.alignments:
    for HSP in aln.hsps:
        if HSP.score > 80 and HSP.expect < 1.0e-10 and HSP.identities/HSP.align_length > 0.80:
            print(aln.accession,HSP.identities/HSP.align_length)
            homologue.append(SeqRecord(Seq(HSP.sbjct), aln.title.split( ">", 1)[0], "", ""))


print("Total:",len(homologue))


SeqIO.write(homologue, os.path.join(wd,"data/homologue/BST2", f"{seq_id}_BST2.fasta"), "fasta")

Accession Identity%
NP_004326 1.0
BAD96844 0.9944444444444445
NP_001266147 0.9567567567567568
XP_018869871 0.9513513513513514
NP_001266560 0.9459459459459459
NP_001177409 0.9459459459459459
ADI58596 0.907608695652174
NP_001295602 0.9017341040462428
BAG64608 0.9629629629629629
NP_001166058 0.9127906976744186
ADI58598 0.9190751445086706
XP_030675274 0.9190751445086706
3MQB_A 1.0
3MQC_A 0.9739130434782609
3MQ7_A 0.9478260869565217
3NWH_A 0.963302752293578
2XG7_A 1.0
XP_024206821 0.92
3MQ9_A 0.9318181818181818
2X7A_A 0.9672131147540983
Total: 20


20